## 1. Data Exploration
### 1.1 Basic Information

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt 
from sklearn import manifold, datasets
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns
import csv
import json


sns.set_style('darkgrid') 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# Any results you write to the current directory are saved as output.
credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
# print(credits.shape) #(4803, 4)

del credits['title']
del movies['id']
full=pd.concat([credits,movies],axis=1)
print(full.shape) #the size of the dataframe

In [ ]:
full.info()  #show the information

In [ ]:
full.head() #get the first 5 lines of the dataframe (full)

### 1.2 Matric Data

In dataframe "full", there are some columns have matric data: movie_id, budget, released date,revenue,runtime,popularity,vote_average and vote_count.In this section, we will do some exploration about these data.

First, we can drop the movies with NaN value of release_date, then use Seaborn to visualize the distribution of the movies with release_time.

In [ ]:
sub0=full[['popularity','vote_average','release_date']]
sub0.release_date=pd.to_datetime(sub0.release_date)
sub0.release_date=sub0.release_date.dt.year
#dorp the movies with NaN value of release_date
sub0 = sub0[~sub0.release_date.isnull()] 
sub0 = sub0[~sub0.popularity.isnull()] 
sub0 = sub0[~sub0.vote_average.isnull()] 
# sub0=sub0[~sub0.popularity.str.isnumeric()]

#float to int 
sub0.release_date = sub0.release_date.apply(int)


plt.figure(figsize=(20, 10))
sns.distplot(sub0.release_date)
plt.show()

Then we use scatter diagram how the popularity and vote_average of movies changed according to the generation.

In [ ]:
sns.relplot(x="release_date", y="vote_average", data= sub0, hue ="popularity",palette="ch:r=-.5,l=.75",alpha=1,height=10)

In [ ]:
# print(sub0.popularity.max())
delete=[875.581305,418.708552,434.278564,481.098624,514.569956,724.247784]
sub0_1=sub0[~sub0['popularity'].isin(delete)]
sns.relplot(x="release_date", y="vote_average", data= sub0_1, hue ="popularity",palette="ch:r=-.5,l=.75",alpha=.4,height=10)

We narrow the scope to the movies after 2000 years.

In [ ]:
sub0_2=sub0[sub0['release_date']>= 2000] #after 2000 years
sub0_2.release_date = sub0_2.release_date.apply(int)
sns.relplot(x="release_date", y="vote_average", data= sub0_2, hue ="popularity",palette='Set3',sizes=(40,1000),alpha=1,height=8)

Then， we want to explore what is the revenue of the movies influence its popularity and vote_average.

In [ ]:
sub1=full[['budget','revenue','popularity','vote_average']]
print(sub1)
plt.figure(figsize=(10, 10))
profit=sub1.revenue.values-sub1.budget.values
vote_average=sub1.vote_average.values
popularity=sub1.popularity.values
print(profit)
print(vote_average)
# plt.scatter(profit, vote_average)
plt.scatter(profit,popularity)
#plt.scatter(sub1.revenue.values,sub1.popularity)
plt.show()

In [ ]:

sub1=sub1[sub1['vote_average']>= 7] #vote>=7
sns.relplot(x="revenue", y="vote_average", data= sub1,size ="popularity",sizes=(40,1000),palette="ch:r=-.5,l=.75",alpha=.4,height=10)

### 1.3 Json Data

We explore the crew data and the keywords data, which may decide what type of the movies.


**The crews' job of the movies: **

In [ ]:
 def getcrew_job(x):
    a=[]
    for i in x:
        a.append(i['job'])
    return a

sub2=full[['crew','vote_average']]
#crew_dict=json.load(sub2.crew.values)
print(type(sub2.crew))
sub2.crew=sub2.crew.apply(json.loads)
crew_job=sub2.crew.apply(getcrew_job)
result = crew_job.apply(pd.value_counts) #result is dataframe
result=result.fillna(0)  #substitute "NaN" to 0
result_top10=result.head(10) #just print the first 10 lines
result_top10['col_sum'] = result_top10.apply(lambda x: x.sum(), axis=1) #the sum of all crews and save it to col_sum
print(result_top10)

In [ ]:
plt.figure(figsize=(50, 30))
sns.heatmap(result_top10, annot=False, fmt=".1f")
plt.show()

However, since the number of dimension is large, the heatmap had bad performance on showing the pattern of different types of movies. So we decide to deduce the dimensions.

**The keywords of the movies:**

In [ ]:
def get_keywords(x):
    a=[]
    for i in x:
        a.append(i['name'])
    return a

sub3=full[['keywords']]
sub3.keywords=sub3.keywords.apply(json.loads)
keywords=sub3.keywords.apply(get_keywords)
result2 = keywords.apply(pd.value_counts) 
# result2.loc['Row_sum'] = result2.apply(lambda x: x.sum())
result2=result2.fillna(0)  #substitute "NaN" to 0
result2['Col_sum'] = result2.apply(lambda x: x.sum(), axis=1)
result_top10_2=result2.head(10)
print(result_top10_2)
print(result_top10_2.keys()) # get the colomn name

## 2.Classify the type of movies
Just combine the crews and the keywords as the features of the classifer.

In [ ]:
sub4=full[['crew','keywords']]
sub4.crew=sub4.crew.apply(json.loads)
sub4.crew=sub4.crew.apply(getcrew_job)
sub4.keywords=sub4.keywords.apply(json.loads)
sub4.keywords=sub4.keywords.apply(get_keywords) # sub4.crew type: series
result4 = sub4.crew.apply(pd.value_counts) #result4 type: dataframe

result4=pd.concat([result4,sub4.keywords.apply(pd.value_counts)]) # combine two dataframes
result4=result4.fillna(0)  
result_top10_4=result4.head(10)
print(result_top10_4)


Then we use TSNE to reduce the dimension to two dimension.

In [ ]:
tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
result4_tsne = tsne.fit_transform(result4)
print("Org data dimension is {}. Embedded data dimension is {}".format(result4.shape[-1], result4_tsne.shape[-1]))

In [ ]:
print(type(result4_tsne))
print(result4_tsne)
np.save("./result4_tsne.npy",result4_tsne) # save as npy file

We use kmeans to cluster.

In [ ]:
d=[]
for i in range(1,40):   #experiements on different value of k
    km=KMeans(n_clusters=i,init='k-means++',n_init=10,max_iter=300,random_state=0)
    km.fit(result4_tsne)
    d.append(km.inertia_)  #inertia:Sum of squares of errors in clusters

plt.plot(range(1,40),d,marker='o')
plt.xlabel('number of clusters')
plt.ylabel('distortions')
plt.show()

In [ ]:
# kmeas clustering
#from sklearn import metrics
result4_kmeans=KMeans(n_clusters=20,random_state=0)
result4_kmeans.fit(result4_tsne)
result4_pre=result4_kmeans.predict(result4_tsne)  #predict

In [ ]:
# visualization of the clustering
centers=result4_kmeans.cluster_centers_ 
print(centers)
colors=['#D0505D','#E1929A','#B4D1D9','#6194A7',
        '#203643','#F6A945','#FEEB90','#Fd8732',
        '#262F34','#6B3A7F',"#006B89","#978065",
        "#3A281C","#9b59b6", "#3498db", "#95a5a6", 
        "#e74c3c", "#34495e", "#2ecc71","#9EC1E0",'#131B1B']
plt.figure(figsize=(10, 6))
for j in range(20):
    index_set=np.where(result4_pre==j)
    cluster=result4_tsne[index_set]
    plt.scatter(cluster[:,0],cluster[:,1],c=colors[j],marker='.')  
    plt.plot(centers[j][0],centers[j][1],'o',markerfacecolor=colors[j],markeredgecolor='k',markersize=8)  #画类别中心
plt.show()

In [ ]:
file_npy='./result4_tsne.npy'
np.load(file_npy)
#print(np)
#plt.plot(re[:,0],re[:,1])
#plt.show()

## 3. Some usage of Seaborn

### 3.1 the color and the style

In [ ]:
current_palette=sns.color_palette() 
#the parameter is the name of the themes, and seaborn has six default themes(deep,muted,pastel,bright,dark,colorblind)
sns.palplot(current_palette)

#use your own color
mycolor=['#D0505D','#E1929A','#B4D1D9','#6194A7',
        '#203643','#F6A945','#FEEB90','#Fd8732']
sns.palplot(sns.color_palette(mycolor))

sns.palplot(sns.color_palette("Blues"))
# reverse:Blues_r 

sns.palplot(sns.cubehelix_palette(8))